In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드 함수
def load_data(pattern, num):
    path = f'algo_dataset/{pattern}/{pattern}_{num}.csv'
    df = pd.read_csv(path)
    return df

# 슬라이딩 윈도우 생성 함수
def create_sliding_window(data, window_size):
    sliding_windows = []
    for i in range(len(data) - window_size + 1):
        sliding_windows.append(data[i:i+window_size])
    return np.array(sliding_windows)

# 패턴 리스트
patterns = ['ascending_triangle', 'descending_triangle', 'ascending_wedge', 'descending_wedge', 'double_top', 'double_bottom']
window_size = 70  # 윈도우 크기 설정
data = []

# 모든 패턴 데이터를 불러오고 슬라이딩 윈도우 적용
for pattern in patterns:
    for i in range(1000):  # 각 패턴에 1000개의 데이터가 있음
        df = load_data(pattern, i+1)
        scaled_data = MinMaxScaler().fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']].values)
        windows = create_sliding_window(scaled_data, window_size)
        data.extend([(window, pattern) for window in windows])

# 데이터를 학습용, 테스트용으로 분리합니다.
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=True)

# 데이터 전처리 함수
def prepare_data(data):
    X, y = [], []
    for window, label in data:
        X.append(window)
        y.append(patterns.index(label))  # 패턴 이름을 숫자로 변환

    X = np.array(X)
    y = tf.keras.utils.to_categorical(y, num_classes=6)  # 6개의 클래스에 대한 one-hot encoding

    return X, y

# 학습 데이터 및 테스트 데이터 전처리
X_train, y_train = prepare_data(train_data)
X_test, y_test = prepare_data(test_data)

# 모델 생성 함수
def create_model():
    model = Sequential()

    # CNN Layers
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(window_size, 5)))  # window_size는 OHLCV 데이터의 길이
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))

    # LSTM Layer
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.3))

    # Fully Connected Layer
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(6, activation='softmax'))  # 6개의 패턴에 대한 확률 출력

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# 모델 생성
model = create_model()

# Early stopping을 사용해 과적합 방지
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train, 
          epochs=50, 
          batch_size=64, 
          validation_data=(X_test, y_test), 
          callbacks=[early_stopping])

# 최종 성능 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Final test accuracy: {accuracy:.4f}")

In [ ]:
import os

# 모델 저장 디렉토리 생성
save_dir = '../save_model'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 모델 저장 경로
model_save_path = os.path.join(save_dir, 'chart_pattern_model.h5')

# 모델 저장
model.save(model_save_path)
print(f"Model saved at {model_save_path}")